# Convert Classic Esri Tabbed StoryMap
Fetch JSON from a Classic Esri Tabbed Story Map and convert each tab into its own ArcGIS StoryMap with the cover supressed. These StoryMaps can then be incoporated into an ArcGIS Collection to replcatee classic app look and feel

In [53]:
# Configure imports and environment variables
#import arcgis
from bs4 import BeautifulSoup
from arcgis.apps.storymap import StoryMap, Sidecar, Text, Image, Map, Embed, Themes
from arcgis.gis import GIS, Item
from IPython.display import display
import pandas as pd
import json, re, requests, sys, time 

from arcgis.apps.storymap import StoryMap

agoNotebook = False

# Set Pandas dataframe display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',1000)

In [2]:
# Print Python and ArcGIS for Python versions
import sys
print(f"Python version: ",sys.version)
import arcgis
print("ArcGIS for Python API version: ",arcgis.__version__)

Python version:  3.11.11 (main, Mar  3 2025, 15:29:37) [MSC v.1938 64 bit (AMD64)]
ArcGIS for Python API version:  2.4.1


In [3]:
# Connect to ArcGIS Online
# Define the GIS
if agoNotebook == False:
    import keyring
    service_name = "system" # Use the default local credential store
    success = False # Set initial state

    # Ask for the username
    while success == False:
        username_for_keyring = input("Enter your ArcGIS Online username:") # If you are using VS Code, the text input dialog box appears at the top of the window
        # Get the credential object
        credential = keyring.get_credential(service_name, username_for_keyring)
        # Check if the username is in the credential store
        if credential is None:
            print(f"'{username_for_keyring}' is not in the local system's credential store. Try another username.")
        # Retrieve the password, login and set the GIS portal
        else:
            password_from_keyring = keyring.get_password("system", username_for_keyring)
            portal_url = 'https://www.arcgis.com'  
            gis = GIS(portal_url, username=username_for_keyring, password=password_from_keyring)
            success = True
            # Print a success message with username and user's organization role
            print("Successfully logged in as: " + gis.properties.user.username, "(role: " + gis.properties.user.role + ")")
else:
    gis = GIS("home")

Successfully logged in as: dasbury_storymaps (role: org_admin)


In [44]:
# Define the Classic StoryMap item id
classic_storymap_id = '597d573e58514bdbbeb53ba2179d2359'
# Fetch the StoryMap Item from AGO
classic_item = Item(gis=gis,itemid=classic_storymap_id)
# Fetch the StoryMap data
classic_data = Item.get_data(classic_item)
if type(classic_data) == dict:
    classic_item_json = json.dumps(classic_data)
    classic_item_data = json.loads(classic_item_json)
else:
    classic_item_data = json.loads(classic_data)

In [96]:
# Helper function to extract HTML side panel text
def parse_html_to_story_content(description_html):
    soup = BeautifulSoup(description_html, "html.parser")
    content_nodes = []
    
    # Parse each image-container div separately, handling nested containers as separate units
    image_containers = soup.find_all("div", class_="image-container")
    print(f"Found {len(image_containers)} image containers")
    processed_containers = set()
    
    for container in image_containers:
        if container in processed_containers:
            continue
        
        # Mark all nested containers inside this one as processed to avoid duplication
        nested_containers = container.find_all("div", class_="image-container")
        processed_containers.update(nested_containers)
        processed_containers.add(container)
        
        # Extract images inside this container
        imgs = container.find_all("img")
        print(f"Found {len(imgs)} images within image container")
        for img in imgs:
            src = img.get("src")
            alt = img.get("alt", "")
            if src:
                img_obj = Image(src)
                img_obj.alt_text = alt
                content_nodes.append(img_obj)
        
        # Extract styled text inside container preserving inline HTML (for rich text)
        # Extract text from the container, joining nested elements with a space
        text_content = container.get_text(separator=' ', strip=True)
        # Use inner HTML excluding images (already processed)
        styles = []
        spans = soup.find_all("span", style=True)
        if spans:
            for span in spans:
                style = span.get("style")
                styles.append(style)
            print(styles)
        # Remove images tags to avoid duplication in text
        for img in imgs:
            img.decompose()
        print(text_content)
        
        # Get remaining inner HTML as string
        inner_html = ''.join(str(e) for e in container.contents).strip()
        if inner_html:
            # Use the raw HTML inside a Text node to keep inline styling
            text_node = Text()
            text_node.text = inner_html
            content_nodes.append(text_node)
    
    # For content outside of image-container divs, parse remaining paragraphs, links, etc.
    # Remove all image-container divs to avoid duplicates
    for img_cont in image_containers:
        img_cont.decompose()
    
    # Parse remaining paragraphs
    para = soup.find_all("p")
    print(f"Found {len(para)} paragraphs outside image containers")
    for p in soup.find_all("p"):
        html_content = str(p)
        print(html_content)
        if html_content:
            text_node = Text()
            text_node.text = html_content
            content_nodes.append(text_node)
            #print(text_node.text)
    
    # Parse standalone links for embedding
    for a in soup.find_all("a"):
        href = a.get("href")
        if href:
            content_nodes.append(Embed(href))
    
    print(f"Found {len(content_nodes)} elements")
    return content_nodes

In [97]:
# Extract entries list
entries = classic_item_data["values"]["story"]["entries"]

# Fetch theme group
theme_group = classic_item_data["values"]["settings"]["theme"]["colors"]["group"]
if theme_group == "dark":
    new_theme = Themes.OBSIDIAN
elif theme_group == "light":
    new_theme = Themes.SUMMIT

created_storymaps = []
loop_limit = 1
for i, entry in enumerate(entries):
    # Create a new StoryMap
    story = StoryMap()
    story.theme(new_theme)

    # Create Sidecar immersive section
    sidecar = Sidecar(style="docked-panel")

    # Add Sidecar to story
    story.add(sidecar)

    # Determine media content for main stage
    media_info = entry.get("media", {})
    media_type = media_info.get("type")

    media_content = None
    if media_type == "webmap":
        webmap_id = media_info.get('webmap', {}).get('id')
        if webmap_id:
            media_content = Map(webmap_id)
    elif media_type == "webpage":
        webpage_url = media_info.get("webpage", {}).get("url")
        if webpage_url:
            media_content = Embed(webpage_url)

    # Create text panel content from description (HTML)
    description_html = entry.get("description", "")
    text_panel = Text(description_html)

    # Convert HTML from side panel to AGSM content items
    narrative_nodes = parse_html_to_story_content(description_html)

    # Add a slide to the sidecar with text panel and main media
    sidecar.add_slide(contents=narrative_nodes, media=media_content)

    # Set webmap properties. Map must be added to the story before setting viewpoint
    if media_type == "webmap":
        # Set the extent for the map stage
        extent_json = media_info.get('webmap', {}).get('extent')
        if extent_json:
            media_content.set_viewpoint(extent=extent_json)  # Extent dict per docs
        # Set layer visibility (if StoryMap Map object supports)
        layers = media_info.get('webmap', {}).get('layers', [])
        if hasattr(media_content, "layers"):
            for lyr in media_content.layers:
                for lyr_json in layers:
                    if lyr.id == lyr_json['id']:
                        lyr.visible = lyr_json['visibility']

    # Hide the cover by setting 'hide_cover' - this is conceptual; actual implementation may vary
    # You may need to modify the story's cover node or settings to hide it as per API capabilities
    story.hide_cover = True  # If available; alternatively configure via story resources/settings

    # Save and publish storymap
    story_title = entry.get("title", "Untitled Story")
    #story.save(title=story_title, tags=["auto-created"], publish=True)

    created_storymaps.append(story)
    print(f"Created replica of {story_title}")
    if i < loop_limit:
        break

print(f"Created {len(created_storymaps)} StoryMaps")

Found 4 image containers
Found 1 images within image container
['color:#E5FA84', 'font-size:20px', 'font-size:12px', 'color:#E2F782', 'font-size:10px']
Hurricane Katrina displaced over one million Louisiana residents -- an estimated 277,000 did not come back to resettle.
Found 0 images within image container
['color:#E5FA84', 'font-size:20px', 'font-size:12px', 'color:#E2F782', 'font-size:10px']

Found 6 paragraphs outside image containers
<p style="text-align:center"><img alt="" height="132" src="https://lh3.googleusercontent.com/-Frhyo5iY8mo/Vc40Ea084sI/AAAAAAAAAEY/r_AIGqcYI9E/s1600/legends-03.png" width="402"/></p>
<p> </p>
<p><span style="font-size:12px">Threatened by one of the most destructive and influential storms in United States history, those living in the path of Hurricane Katrina fled to every state in the country, with many unable to return home to Louisiana after the storm left. This map, which uses data from the 2006 American Community Survey, presents a good, but imper

In [57]:
print(description_html)

<div class="image-container">
<div class="image-container">
<p style="text-align:center"><img alt="" src="https://lh3.googleusercontent.com/-OZs54tCn6mM/VdYjIf-XQ0I/AAAAAAAAAOU/HrFElBu60xw/s1600/legends-08.png" width="496" height="240"></p>
</div>
&nbsp;

<div class="image-container">&nbsp;</div>

<div class="image-container"><span style="color:#E5FA84"><span style="font-size:20px"><strong>Hurricane Katrina displaced over one million Louisiana residents -- an estimated 277,000 did not come back to resettle.</strong>&nbsp;</span></span></div>
</div>

<p style="text-align:center"><img alt="" src="https://lh3.googleusercontent.com/-Frhyo5iY8mo/Vc40Ea084sI/AAAAAAAAAEY/r_AIGqcYI9E/s1600/legends-03.png" width="402" height="132"></p>

<p>&nbsp;</p>

<div>
<p><span style="font-size:12px">Threatened by one of the most destructive and influential storms in United States history, those living in the path of Hurricane Katrina&nbsp;fled to every state in the country, with many&nbsp;unable to return

In [56]:
print(narrative_nodes)

[Image, Image, Text: paragraph, Text: paragraph, Embed: https://www.census.gov/hhes/migration/data/acs/state-to-state.html, Embed: http://www.nhc.noaa.gov/, Embed: http://www.noaa.gov/, Embed: https://www.weather.gov/, Text: paragraph, Text: paragraph, Text: paragraph]


In [ ]:
# Extract HTML from side panel and create story content elements from it
def parse_html_to_story_content(description_html):
    soup = BeautifulSoup(description_html, "html.parser")
    content_nodes = []
    # Parse images
    for img in soup.find_all("img"):
        src = img.get("src")
        alt = img.get("alt", "")
        if src:
            img_obj = Image(src)
            img_obj.alt_text = alt
            content_nodes.append(img_obj)
    # Parse paragraphs & div text
    for p in soup.find_all("p"):
        txt = p.get_text(strip=True)
        if txt:
            content_nodes.append(Text(txt))
    # Parse links for embedding
    for a in soup.find_all("a"):
        href = a.get("href")
        if href:
            content_nodes.append(Embed(href))
    # Fallback for extra descriptive divs/spans etc.
    for div in soup.find_all("div"):
        txt = div.get_text(strip=True)
        if txt and not any(txt == n for n in content_nodes):
            content_nodes.append(Text(txt))
    return content_nodes